In [3]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import spacy
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

讀 epub 檔的準備

In [4]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [5]:
blacklist = [   '[document]',   'noscript', 'header',   'html', 'meta', 'head','input', 'script',   ]

In [6]:
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output

In [7]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    # ttext = thtmltest(chapters)
    return chapters

讀 epub 檔，並將內容放到 LONELY_text 

In [8]:
LONELY_text = []
output = epub2text(r'LONELY.planet.Thailand.epub')
for html in output:
    Data_Dict = defaultdict(list)
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.find_all('p', {'class': ['flush-left','body-text']})            
    for t in text: 
        LONELY_text.append(t.getText())
output = epub2text(r'LONELY.planet.Taiwan.epub')
for html in output:
    Data_Dict = defaultdict(list)
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.find_all('p', {'class': ['flush-left','body-text']})            
    for t in text: 
        LONELY_text.append(t.getText())

FileNotFoundError: [Errno 2] No such file or directory: 'LONELY.planet.Thailand.epub'

建立 LONELY 和 WEB1T 各自的 word_count，以及 LONELY 的例句

In [9]:
# 建立 LONELY 的 wordcount (只有動詞) 
LONELY_wordcount = defaultdict(lambda:0)
# 建立 LONELY 的例句
LONELY_example = defaultdict(lambda:[])
for line in LONELY_text:
    doc = nlp(line)
    for sent in doc.sents:
        for token in sent:
            if token.text.isalpha() and token.pos_ == 'VERB':
                LONELY_wordcount[token.text.lower()]+=1
                LONELY_example[token.text.lower()].append((' '.join([str(token)for token in sent]),token.i - sent.start))
# 將 word count 排序
LONELY_wordcount_sort = sorted(LONELY_wordcount.items(),key=lambda x:-x[1])
# 將例句排序
LONELY_example_sort = dict()
for word in LONELY_example:
    LONELY_example_sort[word] = sorted(LONELY_example[word],key=lambda x:abs(len(x[0])-8))

NameError: name 'defaultdict' is not defined

In [ ]:
LONELY_example_sort['welcome'][:3]

In [ ]:
# 為了判斷 WEB1T 裡面的詞的詞性，先建立詞性表
wordPOSs = defaultdict(lambda:defaultdict(lambda:0))
for line in open('Wiki_pos_count.txt'):
    word,pos,count = line.split('\t')
    wordPOSs[word][pos] = int(count)
wordPOS = dict()
for word in wordPOSs:
    wordPOS[word] = sorted(wordPOSs[word].items(),key=lambda x:-x[1])[0][0]

In [ ]:
# 建立 WEB1T 的 wordcount (只有動詞) 
WEB1T_wordcount = dict()
WEB1T_wordcount_sort = dict()
for line in open('WEB1T.1gm.txt'):
    word,count = line.strip().split()
    count = int(count)
    # 如果這個詞次數最高的詞性是動詞，或是已經在LONELY_wordcount裡面（表示LONELY_wordcount跑Spacy時這個詞被標註為動詞），才算進去
    if (word in wordPOS and wordPOS[word][0]=='V') or word in LONELY_wordcount:
        WEB1T_wordcount[word]=count
WEB1T_wordcount_sort = sorted(WEB1T_wordcount.items(),key=lambda x:-x[1])

建立 LONELY 和 WEB1T 的詞交集（因為如果是其中一邊沒有的詞，計算rankratio沒有意義）

In [ ]:
word_intersection = set(WEB1T_wordcount.keys()).intersection(set(LONELY_wordcount.keys()))

建立 LONELY 和 WEB1T 各自的 word_rank (只排那些在兩者都有的詞)

In [ ]:
# 建立 LONELY 的 word rank
LONELY_rank = dict()
rank=0
for word,count in LONELY_wordcount_sort:
    if word in word_intersection:
        rank+=1
        LONELY_rank[word] = rank

In [ ]:
# 建立 WEB1T 的 word rank
WEB1T_rank = dict()
rank=0
for word,count in WEB1T_wordcount_sort:
    if word in word_intersection:
        rank+=1
        WEB1T_rank[word] = rank

算 Rank Ratio

In [ ]:
# 算每個詞的 rank ratio
word_rankratio = dict()
for word in LONELY_rank:
    word_rankratio[word] = WEB1T_rank[word]/LONELY_rank[word]
# 將詞依照 rank ratio 排序
word_rankratio_sort = sorted(word_rankratio.items(),key=lambda x:-x[1])

In [ ]:
word_rankratio_sort[:10]

看例句

In [ ]:
LONELY_example_sort['tucked'][:5]

看看有哪些詞是 LONELY 裡面有但 WEB1T 裡面沒有的

In [ ]:
in_LONELY_notin_WEB1T = []
for word in LONELY_wordcount:
    if word not in WEB1T_wordcount:
        in_LONELY_notin_WEB1T.append(word)

len(in_LONELY_notin_WEB1T),len(LONELY_wordcount)

In [ ]:
in_LONELY_notin_WEB1T[:10]

把結果印到檔案

In [ ]:
# 兩位有效數字
from math import log10, floor
def round_sig(x, sig=2):
    res = round(x, sig-int(floor(log10(abs(x))))-1)
    return res if res < 10 else int(res)

In [ ]:
import json
# 把 rank ratio 排序結果印到檔案
with open('word_rankratio_sort.txt','w') as out:
    for i,(word,rankratio) in enumerate(word_rankratio_sort):
        out.write('{}\t{}\t{}\t{}\t{}\n'.format(i+1,word,round_sig(rankratio),round_sig(LONELY_rank[word]),round_sig(WEB1T_rank[word])))
# 把例句印到檔案        
with open('example_sent.json','w') as out:
    out.write(json.dumps(LONELY_example_sort))
    

In [ ]:
# def print_spacy_result(sent):
#     print (sent)
#     for token in sent:
#         print (token,token.pos_)

In [ ]:
#                 if token.text.lower() == 'bungalows':
#                     print ('A')
#                     print (line)
#                     print ('B')
#                     print_spacy_result(doc)
#                     print ('++++')